<a href="https://colab.research.google.com/github/Rajdipa31/AirBnB-NYC_Revenue/blob/main/NLP%E2%80%94TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import files

uploaded = files.upload()

Saving Churn Data_KAM - Churned accounts_All_Raw.csv to Churn Data_KAM - Churned accounts_All_Raw.csv


In [5]:
df = pd.read_csv("Churn Data_KAM - Churned accounts_All_Raw.csv")
df.head()

,Customer ID,Company Name,Status,RAM NAME,CSM Name,Seats,Plan Amount,ARR,Created Date,Cancellation Date,...,Customer Classification,Auto Collection,Cancelled In (Timeframe),Churn Response Cohort,Primary Churn Rason,KAM Notes,Churn Reason Cohort,Cancel Reason,transaction failure,Unnamed: 29
0,44cec7ad-5b68-4d94-9f2c-c6af2bf73063,Wellgate,cancelled,NaN,Johanna Navalici,1,35,420,11-Nov-2025,11-Dec-2025,...,Standard,off,Within 30 days,No Auditing Requirement,The customer only wanted to test the platform.,The customer only wanted to test the platform.,No Ongoing Auditing Requirement,NaN,NaN,NaN
1,f0838580-d6f7-4ca5-85e1-7541342f7c60,Climate Edge Consult Sdn Bhd,cancelled,NaN,Aditya Veerabathini,1,12,144,8-Nov-2025,17-Dec-2025,...,Standard,on,Within 30 days,Reason Not Identified,NaN,NaN,Reason Not Identified,not_paid,NaN,NaN
2,eee7c51e-4789-4b88-81da-c5a79319f3ec,Ich Sumardi,cancelled,NaN,Aditya Veerabathini,1,12,144,18-Oct-2025,27-Dec-2025,...,Standard,on,Within 90 days,Reason Not Identified,NaN,NaN,Reason Not Identified,not_paid,NaN,NaN
3,dfd2717e-55aa-49dc-9d21-dc2854c55408,Pro-Form Nutrition Ltd.,cancelled,NaN,Johanna Navalici,1,12,144,16-Oct-2025,16-Nov-2025,...,Standard,on,Within 30 days,Reason Not Identified,NaN,NaN,Reason Not Identified,NaN,NaN,NaN
4,5c530246-42d7-44e3-b590-133c1c7ffcfe,Unifreshfoods,cancelled,NaN,Prathik P,1,35,420,13-Oct-2025,22-Nov-2025,...,Standard,on,Within 30 days,Reason Not Identified,NaN,NaN,Reason Not Identified,not_paid,NaN,5c530246-42d7-44e3-b590-133c1c7ffcfe


In [6]:
# standardize column names
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)

df.columns

Index(['customer_id', 'company_name', 'status', 'ram_name', 'csm_name',
       'seats', 'plan_amount', 'arr', 'created_date', 'cancellation_date',
       'cancellation_month', 'cancellation_year', 'week', 'quarter', 'country',
       'territory', 'plan', 'industry',
       'number_of_days_with_goaudits\n-------------\ncancellation_date_-_create_date',
       'number_of_months_with_goaudits\n-------------\ncancellation_date_-_create_date',
       'customer_classification', 'auto_collection',
       'cancelled_in_(timeframe)', 'churn_response_cohort',
       'primary_churn_rason', 'kam_notes', 'churn_reason_cohort',
       'cancel_reason', 'transaction_failure', 'unnamed:_29'],
      dtype='object')

Create a Unified NLP Text Column

In [7]:
text_cols = [
    "primary_churn_rason",
    "kam_notes",
    "cancel_reason"
]

# keep only columns that actually exist
text_cols = [c for c in text_cols if c in df.columns]

df["churn_text_raw"] = (
    df[text_cols]
    .fillna("")
    .agg(" ".join, axis=1)
)

# remove empty rows
df = df[df["churn_text_raw"].str.strip() != ""]

df[["customer_id", "churn_text_raw"]].head()

,customer_id,churn_text_raw
0,44cec7ad-5b68-4d94-9f2c-c6af2bf73063,The customer only wanted to test the platform....
1,f0838580-d6f7-4ca5-85e1-7541342f7c60,not_paid
2,eee7c51e-4789-4b88-81da-c5a79319f3ec,not_paid
4,5c530246-42d7-44e3-b590-133c1c7ffcfe,not_paid
5,8296a409-1b23-4450-af3c-f87864cb8ccb,not_paid


Packsges for NLP

In [8]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [9]:
nltk.download("stopwords")
nltk.download("wordnet")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    tokens = text.split()
    tokens = [
        lemmatizer.lemmatize(t)
        for t in tokens
        if t not in stop_words and len(t) > 2
    ]
    return " ".join(tokens)

df["churn_text_clean"] = df["churn_text_raw"].apply(clean_text)

df[["churn_text_raw", "churn_text_clean"]].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


,churn_text_raw,churn_text_clean
0,The customer only wanted to test the platform....,customer wanted test platform customer wanted ...
1,not_paid,paid
2,not_paid,paid
4,not_paid,paid
5,not_paid,paid


In [10]:
df["churn_text_clean"].str.len().describe()

,churn_text_clean
count,633.000000
mean,96.496051
std,132.077510
min,4.000000
25%,4.000000
50%,71.000000
75%,127.000000
max,1939.000000


TF-IDF Vectorization

TF-IDF Score

Term Frequency (TF)-Inverse Document FrequencyIDF = TF × IDF

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    min_df=5,
    max_df=0.85,
    ngram_range=(1, 2)
)

X = tfidf.fit_transform(df["churn_text_clean"])
X.shape

(633, 206)

Output of TF-IDF (What ML Sees)

TF-IDF converts text into a matrix like:

Doc	price	payment_failure	trial	support

Doc 1	0.62	0.00	0.12	0.00

Doc 2	0.00	0.71	0.00	0.05

Clustering to Find Churn Themes

In [12]:
from sklearn.cluster import KMeans

k = 5  # you can tune later
kmeans = KMeans(n_clusters=k, random_state=42)
df["churn_cluster"] = kmeans.fit_predict(X)

In [13]:
terms = tfidf.get_feature_names_out()
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]

for i in range(k):
    print(f"\nCluster {i}")
    print([terms[ind] for ind in order_centroids[i][:10]])


Cluster 0
['paid', 'year chargebee', 'would', 'working', 'work', 'year', 'within', 'wish', 'wanted', 'want cancel']

Cluster 1
['tried', 'however', 'anymore', 'different', 'get', 'customer', 'goaudits', 'using', 'need', 'platform']

Cluster 2
['reactivate', 'please', 'subscription', 'customer', 'want', 'invoice', 'paid', 'customer want', 'subscription customer', 'user']

Cluster 3
['customer', 'account', 'platform', 'client', 'need', 'invoice', 'since', 'using', 'year', 'auditing']

Cluster 4
['requested', 'customer requested', 'customer', 'cancelled', 'reactivated', 'subscription', 'account', 'service', 'longer', 'multiple']


Create Business-Friendly Churn Themes

In [14]:
cluster_map = {
    0: "Trial / Testing Only",
    1: "Pricing / Cost",
    2: "Payment Failure",
    3: "No Auditing Requirement",
    4: "Product / Support Issues"
}

df["churn_theme"] = df["churn_cluster"].map(cluster_map)

Quantify Churn Drivers

In [15]:
(
    df["churn_theme"]
    .value_counts(normalize=True)
    .mul(100)
    .round(1)
)

,proportion
churn_theme,
No Auditing Requirement,69.0
Trial / Testing Only,28.4
Product / Support Issues,1.4
Payment Failure,0.9
Pricing / Cost,0.2


In [16]:
final_df = df[
    [
        "customer_id",
        "company_name",
        "arr",
        "churn_theme",
        "churn_text_raw"
    ]
]

final_df.to_csv("churn_nlp_output_2025.csv", index=False)

In [17]:
from google.colab import files

files.download("churn_nlp_output_2025.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>